In [1]:
import numpy as np
import pandas as pd
np.set_printoptions(precision=4)

# Решение задачи Дирихле:
## Известно, что $u(x,y) = 2x^3y^3$
## Оно является решением задачи Дирихле:
### $\frac{\partial^2 u}{\partial x^2} + \frac{\partial^2 u}{\partial y^2} = -(\underbrace{-12xy(x^2+y^2)}_{f(x,y)})$
### $u(0,y) \equiv 0$
### $u(1,y) = 2y^3$
### $u(x,0) \equiv 0$
### $u(x,1) = 2x^3$

In [4]:
f = lambda x,y: -12*x*y*(x*x+y*y)
u1y = lambda y: 2*y**3
ux1 = lambda x: 2*x**3
u_expected = lambda x,y: 2*x**3*y**3

h = 0.2
a_x = 0
b_x = 1
a_y = 0
b_y = 1
N = int((b_x-a_x)/h) + 1 # ширина сетки вместе с граничными узлами
iN = N - 2 # ширина сетки без граничных узлов

6

## Строим матрицу


In [5]:
def init_U():
    U = np.zeros(shape = (N, N))
    for i in range(N):
        U[i, N - 1] = ux1(i*h)
    for j in range(N):
        U[N - 1, j] = u1y(j*h)
    return U
C = np.matrix([[4, -1] + (N-2)*[0]] +
              [ k*[0] + [-1, 4, -1] + (N - 3 - k)*[0] for k in range(0, N - 2)] +
              [(N-2)*[0] + [-1, 4]], dtype=float)
A = np.block([[C, -np.eye(N = N)] + (N-2)*[np.zeros( shape = (N, N) )]] +
              [ k*[np.zeros( shape = (N, N) )] + [-np.eye(N = N), C, -np.eye(N = N)] + (N - 3 - k)*[np.zeros( shape = (N, N) )] for k in range(0, N - 2)] +
              [(N-2)*[np.zeros( shape = (N, N) )] + [-np.eye(N = N), C]])
A = A/h**2

U = init_U()

U_expected = U.copy() # задаём границы
U_expected[1:iN+1,1:iN+1] = np.array([[u_expected(i*h,j*h) for j in range(1, iN+1)] for i in range(1, iN+1)])

F = np.array([[f(i*h, j*h) for j in range(1, iN+1)] for i in range(1,iN+1)])

matrix([[100., -25.,   0., ...,   0.,   0.,   0.],
        [-25., 100., -25., ...,   0.,   0.,   0.],
        [  0., -25., 100., ...,   0.,   0.,   0.],
        ...,
        [  0.,   0.,   0., ..., 100., -25.,   0.],
        [  0.,   0.,   0., ..., -25., 100., -25.],
        [  0.,   0.,   0., ...,   0., -25., 100.]])

In [33]:
iter_method_spectral = np.cos(np.pi * h)
apost_est_coef = iter_method_spectral/(1-iter_method_spectral)
def iteration_method(U, eps = 1e-5):

    init_error = np.max(np.abs(U - U_expected))

    U_reshaped = np.reshape(U, newshape=(N**2, 1)) # преобразуем матрицу сетки к вектору
    AU = A@U_reshaped
    AU_reshaped_cut = np.reshape(AU, newshape=(N, N))[1:iN+1, 1:iN+1]

    init_discrepancy = np.max(np.abs(AU_reshaped_cut - F))
    U_new = U.copy()
    min_iters = -2*np.log(eps)/(np.pi*h)**2
    output_list = list()
    iter = 0
    iter_diffs = [0]*3 # Хранит разницу приближений последних трёх итераций
    rel_discrepancy = 1
    while rel_discrepancy > eps and iter < min_iters:
        for i in range(1, iN+1):
            for j in range(1, iN+1):
                U_new[i, j] = 0.25*(U[i-1, j]+U[i+1,j]+U[i, j-1]+U[i, j+1]+F[i-1,j-1]*h*h)

        iter_diff = np.max(np.abs(U - U_new))

        U = U_new.copy()

        U_reshaped = np.reshape(U, newshape=(N**2, 1)) # преобразуем матрицу сетки к вектору
        AU = A@U_reshaped
        AU_reshaped_cut = np.reshape(AU, newshape=(N, N))[1:iN+1, 1:iN+1]

        error = np.max(np.abs(U - U_expected))
        discrepancy = np.max(np.abs(AU_reshaped_cut - F))
        rel_discrepancy = discrepancy / init_discrepancy
        rel_error = error / init_error

        iter_diffs[iter % 3] = iter_diff
        apost_est = apost_est_coef * iter_diff
        spec_rad_est = ' ' if iter < 2 else np.sqrt( iter_diffs[iter % 3]/ iter_diffs[(iter - 2) % 3] )

        output_list += [[iter + 1, discrepancy, rel_discrepancy,
                         error, rel_error, iter_diff, apost_est, spec_rad_est]]
        iter+=1
    output_list += [[' ', 'min_iters:', int(min_iters), ' ', ' ', ' ', ' ', ' ']]
    return {'solution': U ,'output': pd.DataFrame(data = output_list,
                                                  columns = ['k', 'Curr. discr.', 'Rel. discr.', 'Curr. error', 'Rel. error', '$ \lVert U_k - U_{k-1} \rVert $', 'Apost. est.', '$\rho_k$']).set_index(keys = 'k')}

In [34]:
iter_summary = iteration_method(U)
solU = iter_summary['solution']
iter_summary['output'][::10]

,Curr. discr.,Rel. discr.,Curr. error,Rel. error,$ \lVert U_k - U_{k-1} \rVert $,Apost. est.,$\rho_k$
k,,,,,,,
1,9.5968,0.231977,0.170784,0.325745,0.413696,1.752444,
11,0.29276,0.007077,0.009847,0.018782,0.003855,0.016331,0.783091
21,0.035165,0.000850,0.001183,0.002256,0.000435,0.001844,0.808302
31,0.004224,0.000102,0.000142,0.000271,0.000052,0.000221,0.808999
41,0.000507,0.000012,0.000017,0.000033,0.000006,0.000027,0.809017


In [ ]:
U[0:N + 1,0:N+1]

In [ ]:
F


In [ ]:
solU
